In [25]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import WebBaseLoader
import bs4
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores import FAISS
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
load_dotenv()
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")

In [26]:
loader = WebBaseLoader(web_path=(
    "https://www.parkside.com.br/como-funciona", "https://www.parkside.com.br/diferenciais",
    "https://www.parkside.com.br/duvidas-frequentes"),
    bs_kwargs=dict(parse_only=bs4.SoupStrainer(
        class_=(
            "container-col-18 is-pad-t-72", "container-col-18 less-padding is-pad-tb-72", "faq-question",
            "faq-answer", "faq-answer-p", "faq-question-label no-margin")
    )))
text_documents = loader.load()

In [27]:
text_documents

[Document(page_content='Alugar com a Parkside é fácilSua mudança nunca esteve tão perto!1Selecione seu studioNavegue pela nossas unidades para conferir a localização que mais te interessa e selecione o studio desejado conforme suas necessidadesCONHECER\xa0STUDIOS2Decida quanto tempo quer ficarVocê pode se tornar um Parksider a partir de 1 mês até o tempo que quiser, quanto maior o tempo de locação, melhor o preço fica para você.3Envie os dadosCalma! Não vamos pedir pra você ir ao cartório, assinar documentos, reconhecer firma e outras burocracias. Nosso processo é simples e prático, vamos exigir apenas alguns dados pessoais para efetivar a locação.\u200dOs dados necessários são:- CPF- RG- Nome Completo- Data de Nascimento4Faça as malas e entre!Sua moradia estará disponível 24h após confirmação do procedimento. Você contará com a equipe Parkside durante todo o processo. Rápido, né?!', metadata={'source': 'https://www.parkside.com.br/como-funciona'}),
 Document(page_content='100% mobilia

In [28]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=20)
documents = text_splitter.split_documents(text_documents)

In [29]:
db=Chroma.from_documents(documents, OpenAIEmbeddings())

In [30]:
llm=ChatOpenAI(model='gpt-3.5-turbo')

In [31]:
prompt=ChatPromptTemplate.from_template("""
Responda a pergunta apenas baseado no contexto fornecido.
Pense passo a passo antes de fornecer uma resposta.
Tente nao alterar muito o conteudo no contexto fornecido
<context>
{context}
</context>
Question: {input}""")

In [32]:
document_chain=create_stuff_documents_chain(llm,prompt)

In [33]:
retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001A5EFAA3790>)

In [34]:
##Retriever Chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [35]:
response = retrieval_chain.invoke({'input':"Quanto e 2+ 2"})

In [36]:
response['answer']

'4'